# Setup and connect to iRODS

## The iRODS environment file

To connect to a specific iRODS instance you will need a so-called `irods_environment.json` file. This file contains all parameters to establish the connection. Your iRODS system administrator or service provider will give you such an environment file.

When you are working with Yoda, you can find the information by clicking on the Yoda version label on the bottom of the Yoda portal **after you logged into the portal**.

<img src="img/Yoda_environment.png" align="center" width="200"/>

## Required information to connect to iRODS

The `irods_envirnment.json` needs to contain the following information:

```
{
"irods_host": "<SERVER ADDRESS>",
"irods_port": 1247,
"irods_home": "<YOUR CURRENT WORKING DIRECTORY>",
"irods_user_name": "<YOUR USERNAME>",
"irods_zone_name": "<ZONE NAME>",
"irods_authentication_scheme": "pam",
"irods_encryption_algorithm": "AES-256-CBC",
"irods_encryption_key_size": 32,
"irods_encryption_num_hash_rounds": 16,
"irods_encryption_salt_size": 8,
"irods_client_server_policy": "CS_NEG_REQUIRE",
"irods_client_server_negotiation": "request_server_negotiation"
}
```
Apart from your user name all settings should be given by your iRODS service provider.

**Important**: please make sure that the parameter `"irods_client_server_policy": "CS_NEG_REQUIRE",` is there!

## Additional information

- The `"irods_home"` functions similarly to a "current working directory" and you will be be guided to that location automatically. So we encourage you to set it to a valid location for your project.

    - Plain iRODS: `"irods_home": "/<ZONE NAME>/home/<YOUR USERNAME>"`.

    - Yoda: `"irods_home": "/<ZONE NAME>/home/research-<YOUR GROUP>"`
    

- We also encourage you to ask your iRODS provider about the `"irods_default_resource"`. 
    In most Yoda instances this is `"irods_default_resource": "irodsResc"`.

Both of the settings are not essential but will make it easier and less error-prone when working with data in iRODS. The `"irods_home"` can also be changed later.

## Where to put the irods_environment.json?

All iRODS clients (icommands and APIs) expect the above parameters to be stored in a special folder. This folder is called `.irods` and it lies in your home directory:
- Mac: `/Users/<user>/.irods`
- Linux: `/home/<user>/.irods`
- Windows: `C:\Users\<user>\.irods`

In Mac and Linux you can simply create the folder manually in the file browser.
The `.` shows that it is a hidden folder, which can be quite difficult to create in Windows. Below we provide a python code snippet which creates the folder:

In [ ]:
Path.mkdir

In [ ]:
from pathlib import Path

loc = Path.expanduser(Path('~')).joinpath(".irods")
if not loc.exists():
    loc.mkdir()

Please store your `irods_environment.json` in that folder and make sure that its extension is `json`. 

Again under Windows the text editors usually safe files with the `.txt` extension. So please watch out for this.
Below we provide a code snippet which saves your personal iRODS/Yoda information in the right place. Please adjust:

In [ ]:
import json
env = {
    "irods_host": "<FILL_IN>",
    "irods_port": 1247,
    "irods_home": "<FILL_IN>",
    "irods_user_name": "<FILL_IN>",
    "irods_zone_name": "<FILL_IN>",
    "irods_default_resource": "<FILL_IN>",
    "irods_authentication_scheme": "pam",
    "irods_client_server_policy": "CS_NEG_REQUIRE",
    "irods_client_server_negotiation": "request_server_negotiation",
    "irods_encryption_algorithm": "AES-256-CBC",
    "irods_encryption_key_size": 32,
    "irods_encryption_num_hash_rounds": 16,
    "irods_encryption_salt_size": 8
}
loc = Path.expanduser(Path('~')).joinpath(".irods", "irods_environment.json")
with open(loc, 'w') as write_json:
    json.dump(env, write_json)

If your settings are saved correctly under Windows, you should see this file in `C:\Users\<user>\.irods`.

<img src="img/Save_json.png">

Some last check, if the output is `True` all is setup correctly:

In [ ]:
loc = Path.expanduser(Path('~')).joinpath(".irods", "irods_environment.json")
Path.is_file(loc)

## Connecting to iRODS

### Interactively creating an  iRODS session

When you are working interactively in a jupyter notebook or another environment, we can use the following way of connecting:

In [ ]:
from ibridges.interactive import interactive_auth
session = interactive_auth()

This command reads in the information from your `irods_environment.json`; it tries to find a cached password in the `.irods` folder and asks you for your password, in case there is no cached password. 
Upon successful login, the command/function stores your password in a scrambled way so that the next time you will not be bothered with typing in the password again.

Let's see what happens if we rerun this command. We will first delete the current session and login again:

In [ ]:
session.close()
session = interactive_auth()

You see, that `interactive_auth` still remembers the password from the last time you logged in. **NOTE, that the cached password will expire at some point!**

### Connecting by passing a password

In some programming situations you cannot interactively pass a password. For these cases `iBridges` provides you with another function to connect:

In [ ]:
from ibridges import Session
env_file = Path.expanduser(Path('~')).joinpath(".irods", "irods_environment.json")
password = <YOUR PASSWORD>

session = Session(irods_env_path=env_file, password=password)

## What is this session thing?

The `session` stores all information from your `irods_environment.json`, it provides you with even more information about the iRODS you are connected to and it is needed for all interactions with iRODS.

Let's ave a look:

In [ ]:
print(session.username)
print(session.default_resc) # the resource to which data will be uploaded
print(session.zone) 
print(session.server_version)
print(session.home) # default home for iRODS /zone/home/username

You will see some information about you as a user on iRODS. The last statement shows your current working location on iRODS.
**Note that it is not verified that this location exists!**. Let's have a closer look.

## The iRODS home

Let us verify that the current working location indeed exists:

In [ ]:
from ibridges.path import IrodsPath
irods_path = IrodsPath(session, session.home)
irods_path.collection_exists()

We create an `IrodsPath` which is not a simple text or string, but which is actually connected to iRODS through the `session`. With that we can ask iRODS whether the path indeed is a collection (similar to folder, see chapter about Data).

In case the last command gave you a `False`, do not despair. You can set this parameter with.

In [ ]:
session.home = "<YOUR NEW iRODS PATH>"

Another way to access the current iRODS home is the `~`. Linux and Mac users might know this notation:

In [ ]:
print(IrodsPath(session, "~"))